In [ ]:
import freetype
from PIL import Image, ImageDraw

face = freetype.Face('SourceHanSansSC-Light.otf')
print('NumFaces', face.num_faces)
print('StyleName', face.style_name)
face.set_pixel_sizes(17,15)

FB_W = 16
FB_H = 15
FB_W_HALF = FB_W // 2
OUT_FILE_NAME = 'font16x15.4FN'

HEADER_SIZE = 1
BMP_ROW_SIZE = (FB_W - 1) // 8 + 1
BMP_SIZE = BMP_ROW_SIZE * FB_H
CHAR_DATA_SIZE = BMP_SIZE + HEADER_SIZE
FONT_PAGE_SIZE = CHAR_DATA_SIZE * 256

print('BMP_SIZE', BMP_SIZE)
logData = ''
def log(str):
    global logData
    logData += str + '\n'


def checkWidthEnforcement(ch):
    if (ch == ' '):
        return FB_W_HALF
    return 0

def renderChar(ch):
    face.load_char(ch, flags=freetype.FT_LOAD_TARGET_MONO | freetype.FT_LOAD_RENDER)
    fb = [0] * (FB_W*FB_H)
    g = face.glyph
    offY = FB_H - max(g.bitmap_top, 0) - 1
    offX = max(g.bitmap_left, 0)
    b = g.bitmap
    w = b.width
    totalWidth = offX + w
    fbWidth = min(FB_W, totalWidth)
    if fbWidth <= 0:
        fbWidth = 1
    fbWidth = max(fbWidth, checkWidthEnforcement(ch))
    #if (fbWidth - totalWidth) >= 2:
    #    offX += (fbWidth - totalWidth) // 2
    #    totalWidth = offX + w
    h = b.rows
    offY = min(offY, FB_H - h)
    pitch = b.pitch
    buf = b.buffer
    t = 0
    overflowCnt = 0
    for cy in range(0, h):
        for cx in range(0, w):
            if (cx % 8 == 0):
                t = buf[pitch * cy + cx // 8]
            v = 0
            if (t & 0x80):
                v = 1
            posX = offX + cx
            posY = offY + cy
            if (posX < 0) or (posX >= FB_W) or (posY < 0) or (posY >= FB_H):
                overflowCnt += 1
                if overflowCnt <= 2:
                    log('overflow %d %d %s' % ( posX, posY, ch))
            else:
                fb[posX + posY * FB_W] = v
            t <<= 1
    return (fb, fbWidth)

def printFB(fb, fbWidth):
    ret = ''
    for y in range(0, FB_H):
        for x in range(0, fbWidth):
            ret += ['.', '#'][fb[x + y * FB_W]]
        ret += '\n'
    print(ret)
    print(fbWidth)



In [ ]:
logData = ''
fontMagic = b'44Font0000000' + bytes([FB_W, FB_H, BMP_SIZE])
fontIndex = [0xFF] * 256
fontBuf = b''
fontBufPtr = 0

PLANES = [(0x00, 0xFF)] #[(0, 0), (0x30, 0x30), ]
for (l, r) in PLANES:
    for p in range(l, l + r + 1):
        validCharCount = 0
        pageBuf = b''
        for rune in range(p << 8, (p << 8) + 256):
            header = 0
            bmp = [0] * (BMP_SIZE)
            if face.get_char_index(rune) != 0:
                validCharCount += 1
                log(chr(rune))
            (fb, fbWidth) = renderChar(chr(rune))
            header = fbWidth
            for y in range(0, FB_H):
                for x in range(0, FB_W):
                    if fb[y * FB_W + x]:
                        bmp[BMP_ROW_SIZE * y + (x // 8)] |= 1 << (x % 8)
            pageBuf += bytes([header])
            pageBuf += bytes(bmp)
        log('0x%02x00 %d %d' % (p, validCharCount, len(pageBuf)))
        if validCharCount == 0:
            continue
        fontIndex[p] = fontBufPtr
        fontBuf += pageBuf
        fontBufPtr += 1

with open(OUT_FILE_NAME, 'wb') as f:
    f.write(fontMagic)
    f.write(bytes(fontIndex))
    f.write(fontBuf)

with open('log.txt', 'w', encoding='utf8') as f:
    f.write(logData)

In [ ]:
fb, fbWidth = renderChar('测')

printFB(fb, fbWidth)

In [ ]:
txt = """CJK字体测试。

我能吞下玻璃而不伤身体。
我能吞下玻璃而不傷身體。 
私はガラスを食べられます。それは私を傷つけません。
나는 유리를 먹을 수 있어요. 그래도 아프지 않아요

"""
img = Image.new('RGB', (320,240), color='white')
pixels = img.load()
posX = 0
posY = 0

def drawFB(fb, fbWidth):
    global posX, posY

    if (posX + fbWidth >= 320):
        posX = 0
        posY += FB_H + 1
    for y in range(0, FB_H):
        for x in range(0, fbWidth):
            if fb[x + y * FB_W]:
                pixels[posX + x, posY + y] = (0,0,0)
    posX += fbWidth

for ch in txt:
    if (ch == '\n'):
        posX = 0
        posY += FB_H + 1
        continue
    fb, fbWidth = renderChar(ch)
    drawFB(fb, fbWidth)
img